# Fixed Complete MoS2 Analysis Pipeline - All 3 Stages

## This notebook contains the complete, working implementation:
- **Stage 1**: Flake detection using threshold < 140 (optimized for your images)
- **Stage 2**: Multilayer structure detection within flakes  
- **Stage 3**: Twist angle calculation between layers

## All bugs fixed:
- ✅ OpenCV contourArea error fixed
- ✅ Pipeline variable properly defined
- ✅ Complete class implementation included
- ✅ All imports and dependencies included

In [ ]:
# Install required packages
!pip install opencv-python-headless matplotlib numpy scipy scikit-image

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
from scipy import ndimage
from skimage import measure, morphology, filters
import os

# Create directories
os.makedirs('/content/images', exist_ok=True)
os.makedirs('/content/results', exist_ok=True)
os.makedirs('/content/debug', exist_ok=True)

print("✓ Environment setup complete")

In [ ]:
class CompleteMoS2Pipeline:
    def __init__(self):
        self.intensity_threshold = 140  # Optimized based on your results
        self.min_flake_area = 200      # Increased to reduce noise
        self.max_flake_area = 15000    # Reasonable upper limit
        
    def stage1_detect_flakes(self, image_path):
        """Stage 1: Detect MoS2 flakes using optimized intensity threshold"""
        print(f"\n=== STAGE 1: FLAKE DETECTION ===")
        print(f"Processing: {Path(image_path).name}")
        
        # Load image
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
        
        print(f"Image intensity range: {gray.min()} - {gray.max()}")
        
        # Apply optimized threshold (flakes are darker than background)
        binary = (gray < self.intensity_threshold).astype(np.uint8) * 255
        
        # Clean up binary mask
        kernel_open = np.ones((2,2), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel_open)
        
        kernel_close = np.ones((4,4), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel_close)
        
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Filter and analyze flakes
        flakes = []
        for contour in contours:
            area = cv2.contourArea(contour)
            
            # Size filtering
            if area < self.min_flake_area or area > self.max_flake_area:
                continue
            
            # Shape analysis
            perimeter = cv2.arcLength(contour, True)
            if perimeter == 0:
                continue
                
            # Approximate contour
            epsilon = 0.02 * perimeter
            approx = cv2.approxPolyDP(contour, epsilon, True)
            
            # Calculate shape metrics
            hull = cv2.convexHull(contour)
            hull_area = cv2.contourArea(hull)
            solidity = area / hull_area if hull_area > 0 else 0
            
            # Aspect ratio
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = max(w, h) / min(w, h) if min(w, h) > 0 else 1
            
            # Circularity
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            
            # Filter by shape (more permissive for MoS2 flakes)
            is_valid_flake = (
                0.3 < solidity < 1.0 and      # Allow some irregularity
                aspect_ratio < 5.0 and        # Not too elongated
                circularity > 0.15 and        # Not too thin/linear
                3 <= len(approx) <= 10        # Polygonal shape
            )
            
            if is_valid_flake:
                # Calculate centroid
                M = cv2.moments(contour)
                cx = int(M['m10']/M['m00']) if M['m00'] != 0 else 0
                cy = int(M['m01']/M['m00']) if M['m00'] != 0 else 0
                
                flakes.append({
                    'id': len(flakes) + 1,
                    'contour': contour,
                    'approx': approx,
                    'area': area,
                    'perimeter': perimeter,
                    'solidity': solidity,
                    'aspect_ratio': aspect_ratio,
                    'circularity': circularity,
                    'vertices': len(approx),
                    'centroid': (cx, cy),
                    'bbox': (x, y, w, h)
                })
        
        print(f"Stage 1 complete: Found {len(flakes)} valid flakes")
        return img_rgb, gray, binary, flakes
    
    def stage2_detect_multilayers(self, img_rgb, gray, flakes):
        """Stage 2: Detect multilayer structures within flakes"""
        print(f"\n=== STAGE 2: MULTILAYER DETECTION ===")
        
        multilayer_flakes = []
        
        for flake in flakes:
            try:
                # Extract region of interest around the flake
                x, y, w, h = flake['bbox']
                
                # Expand ROI slightly
                margin = 10
                x1 = max(0, x - margin)
                y1 = max(0, y - margin)
                x2 = min(img_rgb.shape[1], x + w + margin)
                y2 = min(img_rgb.shape[0], y + h + margin)
                
                roi_gray = gray[y1:y2, x1:x2]
                roi_rgb = img_rgb[y1:y2, x1:x2]
                
                # Create mask for this flake
                mask = np.zeros(gray.shape, dtype=np.uint8)
                cv2.fillPoly(mask, [flake['contour']], 255)
                roi_mask = mask[y1:y2, x1:x2]
                
                # Look for internal structures using multiple approaches
                internal_structures = self.find_internal_structures(roi_gray, roi_mask, x1, y1)
                
                if internal_structures:
                    flake['internal_structures'] = internal_structures
                    flake['is_multilayer'] = True
                    flake['layer_count'] = len(internal_structures) + 1  # +1 for base layer
                    multilayer_flakes.append(flake)
                else:
                    flake['is_multilayer'] = False
                    flake['layer_count'] = 1
                    
            except Exception as e:
                print(f"Error processing flake {flake['id']}: {str(e)}")
                flake['is_multilayer'] = False
                flake['layer_count'] = 1
                continue
        
        print(f"Stage 2 complete: Found {len(multilayer_flakes)} multilayer structures")
        return multilayer_flakes
    
    def find_internal_structures(self, roi_gray, roi_mask, offset_x, offset_y):
        """Find internal structures within a flake ROI - FIXED VERSION"""
        internal_structures = []
        
        try:
            # Method 1: Edge detection within the flake
            edges = cv2.Canny(roi_gray, 30, 90)
            edges = cv2.bitwise_and(edges, roi_mask)
            
            # Dilate edges slightly to connect gaps
            kernel = np.ones((2,2), np.uint8)
            edges = cv2.dilate(edges, kernel, iterations=1)
            
            # Find internal contours
            contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            
            for contour in contours:
                if len(contour) < 3:  # Skip invalid contours
                    continue
                    
                area = cv2.contourArea(contour)
                
                # Check if this internal structure is significant
                roi_area = np.sum(roi_mask > 0)  # FIXED: count non-zero pixels properly
                area_ratio = area / roi_area if roi_area > 0 else 0
                
                if area > 100 and area_ratio > 0.05:  # At least 5% of the flake area
                    # Adjust coordinates back to full image
                    adjusted_contour = contour + [offset_x, offset_y]
                    
                    # Check if it's roughly triangular
                    epsilon = 0.03 * cv2.arcLength(contour, True)
                    approx = cv2.approxPolyDP(contour, epsilon, True)
                    
                    if 3 <= len(approx) <= 7:  # Roughly polygonal
                        internal_structures.append({
                            'contour': adjusted_contour,
                            'approx': approx + [offset_x, offset_y],
                            'area': area,
                            'vertices': len(approx),
                            'detection_method': 'edge'
                        })
            
            # Method 2: Intensity-based detection of darker regions within flake
            masked_roi = cv2.bitwise_and(roi_gray, roi_mask)
            if masked_roi.max() > 0:
                # Find darker regions within the flake
                mask_pixels = masked_roi[roi_mask > 0]
                if len(mask_pixels) > 0:  # FIXED: Check if we have valid pixels
                    mean_intensity = mask_pixels.mean()
                    dark_threshold = mean_intensity - 15  # Darker threshold for better detection
                    
                    dark_regions = (masked_roi < dark_threshold) & (roi_mask > 0)
                    dark_regions = dark_regions.astype(np.uint8) * 255
                    
                    # Clean up dark regions
                    kernel = np.ones((3,3), np.uint8)
                    dark_regions = cv2.morphologyEx(dark_regions, cv2.MORPH_OPEN, kernel)
                    
                    # Find contours of dark regions
                    dark_contours, _ = cv2.findContours(dark_regions, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    
                    for contour in dark_contours:
                        if len(contour) < 3:  # Skip invalid contours
                            continue
                            
                        area = cv2.contourArea(contour)
                        if area > 150:  # Minimum size for internal structure
                            # Check if not already found by edge method
                            adjusted_contour = contour + [offset_x, offset_y]
                            
                            # Simple duplicate check
                            is_duplicate = False
                            for existing in internal_structures:
                                if abs(existing['area'] - area) < 50:  # Similar area
                                    is_duplicate = True
                                    break
                            
                            if not is_duplicate:
                                epsilon = 0.03 * cv2.arcLength(contour, True)
                                approx = cv2.approxPolyDP(contour, epsilon, True)
                                
                                if len(approx) >= 3:  # Valid polygon
                                    internal_structures.append({
                                        'contour': adjusted_contour,
                                        'approx': approx + [offset_x, offset_y],
                                        'area': area,
                                        'vertices': len(approx),
                                        'detection_method': 'intensity'
                                    })
        
        except Exception as e:
            print(f"Error in find_internal_structures: {str(e)}")
            return []
        
        return internal_structures
    
    def stage3_calculate_twist_angles(self, multilayer_flakes):
        """Stage 3: Calculate twist angles for bilayer structures"""
        print(f"\n=== STAGE 3: TWIST ANGLE CALCULATION ===")
        
        angle_results = []
        
        for flake in multilayer_flakes:
            if not flake['is_multilayer'] or not flake.get('internal_structures', []):
                continue
            
            try:
                # Calculate main flake orientation
                main_vertices = flake['approx'].reshape(-1, 2)
                main_angle = self.calculate_triangle_orientation(main_vertices)
                
                # Calculate angles for internal structures
                twist_measurements = []
                
                for internal in flake['internal_structures']:
                    if len(internal['approx']) >= 3:
                        internal_vertices = internal['approx'].reshape(-1, 2)
                        internal_angle = self.calculate_triangle_orientation(internal_vertices)
                        
                        # Calculate twist angle
                        twist_angle = abs(main_angle - internal_angle)
                        
                        # Normalize to 0-60 degrees (considering 3-fold symmetry of MoS2)
                        twist_angle = min(twist_angle, 180 - twist_angle)
                        if twist_angle > 60:
                            twist_angle = 120 - twist_angle
                        
                        twist_measurements.append({
                            'internal_angle': internal_angle,
                            'twist_angle': abs(twist_angle),
                            'internal_area': internal['area']
                        })
                
                if twist_measurements:
                    angle_results.append({
                        'flake_id': flake['id'],
                        'main_angle': main_angle,
                        'twist_measurements': twist_measurements,
                        'average_twist': np.mean([t['twist_angle'] for t in twist_measurements]),
                        'area': flake['area'],
                        'centroid': flake['centroid']
                    })
                    
            except Exception as e:
                print(f"Error calculating angles for flake {flake['id']}: {str(e)}")
                continue
        
        print(f"Stage 3 complete: Calculated twist angles for {len(angle_results)} bilayer structures")
        return angle_results
    
    def calculate_triangle_orientation(self, vertices):
        """Calculate the orientation angle of a triangular shape"""
        if len(vertices) < 3:
            return 0
        
        # Calculate centroid
        centroid = np.mean(vertices, axis=0)
        
        # Find the vertex furthest from centroid (likely the apex)
        distances = np.linalg.norm(vertices - centroid, axis=1)
        apex_idx = np.argmax(distances)
        apex = vertices[apex_idx]
        
        # Calculate angle from centroid to apex
        angle = np.arctan2(apex[1] - centroid[1], apex[0] - centroid[0])
        return np.degrees(angle) % 360
    
    def visualize_complete_results(self, img_rgb, flakes, multilayer_flakes, angle_results):
        """Visualize all three stages of analysis"""
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Stage 1: All detected flakes
        stage1_img = img_rgb.copy()
        for flake in flakes:
            cv2.drawContours(stage1_img, [flake['contour']], -1, (255, 0, 0), 2)
            cx, cy = flake['centroid']
            cv2.putText(stage1_img, str(flake['id']), (cx-10, cy), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
        
        axes[0,0].imshow(stage1_img)
        axes[0,0].set_title(f'Stage 1: All Flakes ({len(flakes)})')
        axes[0,0].axis('off')
        
        # Stage 2: Multilayer structures
        stage2_img = img_rgb.copy()
        for flake in multilayer_flakes:
            # Draw main flake in green
            cv2.drawContours(stage2_img, [flake['contour']], -1, (0, 255, 0), 2)
            
            # Draw internal structures in cyan
            for internal in flake.get('internal_structures', []):
                cv2.drawContours(stage2_img, [internal['contour']], -1, (0, 255, 255), 2)
            
            cx, cy = flake['centroid']
            cv2.putText(stage2_img, f"{flake['id']}({flake['layer_count']}L)", (cx-15, cy), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        
        axes[0,1].imshow(stage2_img)
        axes[0,1].set_title(f'Stage 2: Multilayer Structures ({len(multilayer_flakes)})')
        axes[0,1].axis('off')
        
        # Stage 3: Twist angles
        stage3_img = img_rgb.copy()
        for result in angle_results:
            # Find corresponding flake
            flake = next((f for f in multilayer_flakes if f['id'] == result['flake_id']), None)
            if flake:
                cv2.drawContours(stage3_img, [flake['contour']], -1, (255, 165, 0), 2)
                
                cx, cy = result['centroid']
                angle_text = f"{result['average_twist']:.1f}°"
                cv2.putText(stage3_img, angle_text, (cx-20, cy+15), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
        
        axes[1,0].imshow(stage3_img)
        axes[1,0].set_title(f'Stage 3: Twist Angles ({len(angle_results)} bilayers)')
        axes[1,0].axis('off')
        
        # Summary statistics
        axes[1,1].axis('off')
        
        summary_text = f"""ANALYSIS SUMMARY
        
Stage 1: Flake Detection
• Total flakes detected: {len(flakes)}
• Intensity threshold: < {self.intensity_threshold}
• Size range: {self.min_flake_area}-{self.max_flake_area} px

Stage 2: Multilayer Detection
• Multilayer flakes: {len(multilayer_flakes)}
• Detection rate: {len(multilayer_flakes)/len(flakes)*100:.1f}%

Stage 3: Twist Angle Analysis
• Bilayer structures: {len(angle_results)}
"""
        
        if angle_results:
            all_angles = [r['average_twist'] for r in angle_results]
            summary_text += f"""• Mean twist angle: {np.mean(all_angles):.1f}°
• Angle range: {np.min(all_angles):.1f}° - {np.max(all_angles):.1f}°
• Standard deviation: {np.std(all_angles):.1f}°"""
        
        axes[1,1].text(0.05, 0.95, summary_text, transform=axes[1,1].transAxes, 
                      verticalalignment='top', fontsize=10, fontfamily='monospace')
        
        plt.tight_layout()
        return fig
    
    def process_complete_pipeline(self, image_path):
        """Run the complete 3-stage analysis pipeline"""
        print(f"\n{'='*60}")
        print(f"COMPLETE MoS2 ANALYSIS PIPELINE")
        print(f"Image: {Path(image_path).name}")
        print(f"{'='*60}")
        
        # Stage 1: Detect flakes
        img_rgb, gray, binary, flakes = self.stage1_detect_flakes(image_path)
        
        # Stage 2: Find multilayer structures
        multilayer_flakes = self.stage2_detect_multilayers(img_rgb, gray, flakes)
        
        # Stage 3: Calculate twist angles
        angle_results = self.stage3_calculate_twist_angles(multilayer_flakes)
        
        # Visualize complete results
        fig = self.visualize_complete_results(img_rgb, flakes, multilayer_flakes, angle_results)
        
        return {
            'image': img_rgb,
            'flakes': flakes,
            'multilayer_flakes': multilayer_flakes,
            'angle_results': angle_results,
            'figure': fig
        }

# Initialize the complete pipeline - FIXED: Properly define the variable
pipeline = CompleteMoS2Pipeline()
print("✓ Complete MoS2 Analysis Pipeline initialized")
print(f"✓ Optimized for intensity threshold < {pipeline.intensity_threshold}")

In [ ]:
# Run complete pipeline on all images
image_dir = Path('/content/images')
image_extensions = ['.jpg', '.jpeg', '.png', '.tiff', '.bmp']

image_files = []
for ext in image_extensions:
    image_files.extend(image_dir.glob(f'*{ext}'))
    image_files.extend(image_dir.glob(f'*{ext.upper()}'))

print(f"Found {len(image_files)} images to process\n")

all_results = []

for image_path in image_files:
    try:
        # Run complete pipeline
        results = pipeline.process_complete_pipeline(str(image_path))
        
        # Save visualization
        plt.figure(results['figure'].number)
        plt.savefig(f'/content/results/{image_path.stem}_complete_analysis.png', 
                   dpi=300, bbox_inches='tight')
        plt.show()
        
        # Print detailed results
        print(f"\n{'='*40}")
        print(f"DETAILED RESULTS")
        print(f"{'='*40}")
        
        flakes = results['flakes']
        multilayer_flakes = results['multilayer_flakes']
        angle_results = results['angle_results']
        
        print(f"\nFLAKE SUMMARY:")
        for flake in flakes:
            layer_info = f" ({flake['layer_count']}L)" if flake.get('is_multilayer') else " (1L)"
            print(f"  Flake {flake['id']}: Area={flake['area']:.0f}px, "
                  f"Vertices={flake['vertices']}, Solidity={flake['solidity']:.2f}{layer_info}")
        
        if angle_results:
            print(f"\nTWIST ANGLES:")
            for result in angle_results:
                print(f"  Flake {result['flake_id']}: {result['average_twist']:.1f}° "
                      f"(from {len(result['twist_measurements'])} measurements)")
                for i, measurement in enumerate(result['twist_measurements']):
                    print(f"    Layer {i+1}: {measurement['twist_angle']:.1f}° "
                          f"(area: {measurement['internal_area']:.0f}px)")
        
        # Save detailed JSON results
        json_data = {
            'filename': image_path.name,
            'analysis_summary': {
                'total_flakes': len(flakes),
                'multilayer_flakes': len(multilayer_flakes),
                'bilayer_structures': len(angle_results)
            },
            'flake_details': [],
            'twist_angle_data': []
        }
        
        # Save flake details
        for flake in flakes:
            flake_data = {
                'id': flake['id'],
                'area': float(flake['area']),
                'vertices': int(flake['vertices']),
                'solidity': float(flake['solidity']),
                'aspect_ratio': float(flake['aspect_ratio']),
                'is_multilayer': bool(flake.get('is_multilayer', False)),
                'layer_count': int(flake.get('layer_count', 1)),
                'centroid': flake['centroid']
            }
            json_data['flake_details'].append(flake_data)
        
        # Save twist angle data
        for result in angle_results:
            angle_data = {
                'flake_id': result['flake_id'],
                'main_angle': float(result['main_angle']),
                'average_twist': float(result['average_twist']),
                'individual_measurements': [
                    {
                        'twist_angle': float(m['twist_angle']),
                        'internal_area': float(m['internal_area'])
                    } for m in result['twist_measurements']
                ]
            }
            json_data['twist_angle_data'].append(angle_data)
        
        # Save to file
        with open(f'/content/results/{image_path.stem}_complete_results.json', 'w') as f:
            json.dump(json_data, f, indent=2)
        
        all_results.append(json_data)
        print(f"\n✓ Complete results saved for {image_path.name}")
        
    except Exception as e:
        print(f"❌ Error processing {image_path.name}: {str(e)}")
        import traceback
        traceback.print_exc()
        continue

# Final summary
if all_results:
    print(f"\n{'='*60}")
    print(f"FINAL SUMMARY - ALL IMAGES")
    print(f"{'='*60}")
    
    total_flakes = sum(r['analysis_summary']['total_flakes'] for r in all_results)
    total_multilayer = sum(r['analysis_summary']['multilayer_flakes'] for r in all_results)
    total_bilayer = sum(r['analysis_summary']['bilayer_structures'] for r in all_results)
    
    print(f"Images processed: {len(all_results)}")
    print(f"Total flakes detected: {total_flakes}")
    print(f"Total multilayer structures: {total_multilayer}")
    print(f"Total bilayer structures with twist angles: {total_bilayer}")
    
    if total_flakes > 0:
        print(f"Multilayer detection rate: {total_multilayer/total_flakes*100:.1f}%")
    
    # Collect all twist angles
    all_twist_angles = []
    for result in all_results:
        for angle_data in result['twist_angle_data']:
            all_twist_angles.append(angle_data['average_twist'])
    
    if all_twist_angles:
        print(f"\nTwist angle statistics:")
        print(f"  Total measurements: {len(all_twist_angles)}")
        print(f"  Mean: {np.mean(all_twist_angles):.1f}°")
        print(f"  Median: {np.median(all_twist_angles):.1f}°")
        print(f"  Range: {np.min(all_twist_angles):.1f}° - {np.max(all_twist_angles):.1f}°")
        print(f"  Standard deviation: {np.std(all_twist_angles):.1f}°")
    
    # Save overall summary
    with open('/content/results/complete_pipeline_summary.json', 'w') as f:
        json.dump(all_results, f, indent=2)
    
    print(f"\n✓ Complete pipeline results saved to /content/results/")
else:
    print("No images were successfully processed.")

## Results Interpretation Guide

### Output Files:
- **`[filename]_complete_analysis.png`**: Visual summary of all 3 stages
- **`[filename]_complete_results.json`**: Detailed numerical data
- **`complete_pipeline_summary.json`**: Summary of all processed images

### Stage Results:
1. **Stage 1 (Blue outlines)**: All detected MoS2 flakes
2. **Stage 2 (Green/Cyan outlines)**: Multilayer structures with internal features
3. **Stage 3 (Orange outlines + angles)**: Bilayer twist angles in degrees

### Key Metrics:
- **Layer count**: Number of layers detected (1L, 2L, 3L, etc.)
- **Twist angles**: Rotation between overlapping triangular layers
- **Detection rate**: Percentage of flakes showing multilayer structure

### Quality Indicators:
- **Good detection**: Clear triangular outlines, reasonable twist angles (0-60°)
- **False positives**: Very small areas, irregular shapes, extreme angles
- **Missed structures**: Manual inspection may reveal additional multilayer regions

### Next Steps:
If results need refinement, adjust these parameters in the code:
- `intensity_threshold` (currently 140)
- `min_flake_area` (currently 200)
- Internal structure detection sensitivity
